In [204]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os

In [205]:
# function to create soup
def create_soup(url):
    response = requests.get(url)
    soup     = BeautifulSoup(response.content, 'html.parser')
    return soup


In [206]:
url = 'https://www.hkex.com.hk/eng/stat/smstat/dayquot/d220608e.htm#dealings_suspstocks'

# creating soup
soup = create_soup(url)

In [207]:
# required texts are inside 'font' tags
# getting all texts
all_texts = soup.find_all('font', attrs = {'size' : '1'})

In [208]:
# let's check type of it
type(all_texts)

bs4.element.ResultSet

In [209]:
# let's check length of it
len(all_texts)

814

In [210]:
# slicing QUOTATIONS from all_texts
quotations = all_texts[:112]

In [211]:
# try len(quotations)
# try quotations[0]
# try quotations[-1]

In [212]:
# first item and last item of the quotations list contain extra information
# we will need to process these items
quotations[0].text.split('QUOTATIONS')[-1]

"\r\n\r\n CODE  NAME OF STOCK    CUR PRV.CLO./    ASK/    HIGH/      SHARES TRADED/\r\n                            CLOSING      BID     LOW        TURNOVER ($)\r\n\r\n     1 CKH HOLDINGS     HKD   54.85    54.35    55.40            5,234,462\r\n                              54.35    54.30    54.25          286,201,169\r\n     2 CLP HOLDINGS     HKD   76.95    76.55    77.30            3,374,157\r\n                              76.50    76.50    76.40          258,528,175\r\n     3 HK & CHINA GAS   HKD    8.74     8.53     8.76           30,744,940\r\n                               8.52     8.52     8.51          264,009,852\r\n     4 WHARF HOLDINGS   HKD   25.15    25.25    25.40              683,860\r\n                              25.25    25.20    24.95           17,249,090\r\n     5 HSBC HOLDINGS    HKD   51.75    51.30    52.10           20,744,261\r\n                              51.25    51.25    51.00        1,069,075,134\r\n     6 POWER ASSETS     HKD   51.35    51.15    51.70

In [213]:
# something unusual
quotations[2].text.split('\r\n')[-1]

''

In [214]:
# desired headings
headings = quotations[0].text.split('QUOTATIONS')[-1].split('\r\n\r\n')[1]
headings

' CODE  NAME OF STOCK    CUR PRV.CLO./    ASK/    HIGH/      SHARES TRADED/\r\n                            CLOSING      BID     LOW        TURNOVER ($)'

In [215]:
# create a list for all quotation
quotations_lists = []

# processing of first item
quotation_first_item = quotations[0].text.split('QUOTATIONS')[-1].split('\r\n\r\n')[2]

# adding first item in quotations_lists
quotations_lists += quotation_first_item.split('\r\n')[:-1]

# adding remaining items in quotations_lists except last one
for item in quotations[1:111]:
    quotations_lists += item.text.split('\r\n')[:-1]

# processing of last item
quotation_last_item = quotations[111].text.split('\r\n-------------------------------------------------------------------------------')[0]

# adding last item in quotations_lists
quotations_lists += quotation_last_item.split('\r\n')

In [216]:
len(quotations_lists)

33356

In [217]:
quotations_lists[0:10]

['     1 CKH HOLDINGS     HKD   54.85    54.35    55.40            5,234,462',
 '                              54.35    54.30    54.25          286,201,169',
 '     2 CLP HOLDINGS     HKD   76.95    76.55    77.30            3,374,157',
 '                              76.50    76.50    76.40          258,528,175',
 '     3 HK & CHINA GAS   HKD    8.74     8.53     8.76           30,744,940',
 '                               8.52     8.52     8.51          264,009,852',
 '     4 WHARF HOLDINGS   HKD   25.15    25.25    25.40              683,860',
 '                              25.25    25.20    24.95           17,249,090',
 '     5 HSBC HOLDINGS    HKD   51.75    51.30    52.10           20,744,261',
 '                              51.25    51.25    51.00        1,069,075,134']

In [218]:
def splited_list(item):

    ### different types of item

    # '50090#UB#MTUANRP2303N  HKD TRADING SUSPENDED'
    # '40808 SHINSUN N2308    USD TRADING SUSPENDED'
    # '709#GIORDANO INT'L   HKD TRADING HALTED'
    # '50234#UB#JDCOMRP2302B  HKD    0.04     0.015    0.027           1,312,500'
    # '50076 CS#MTUANRC2302M  HKD    0.155    0.175    0.176             175,000'
    # '50062#BP#HSI  RP2210H  HKD    0.053    0.02     0.024          44,340,000'
    # '40101 KM RT B2412      USD     N/A      -        -                      -'  
    # '122 CROCODILE        HKD    0.34     0.39     0.405          12,245,000'
    # '*  388 HKEX             HKD  352.20   361.40   362.80            8,347,757' 

    elements = item.split()
    li = []
    for elem in elements:
        if '#' in elem:
            li += elem.split('#')
        else:
            li.append(elem)
    return li


def suspended(li):
    # sample list
    # li = ['50105', 'UB', 'HSI', 'RP2211R', 'HKD', 'TRADING', 'SUSPENDED']

    code = int(li[0])
    nos  = ' '.join(li[1:-3])      # nos = Name of Stock
    cur  = li[-3]

    return code, nos, cur


def check(x):
    try:
        elem = float(x)
    except:
        elem = x
    return elem

def check_lsit_last_item(x):
    try:
        elem = int(''.join(x.split(',')))
    except:
        elem = x
    return elem

def not_suspended(li_1, li_2):
    # sample list
    # li = ['50234', 'UB', 'JDCOMRP2302B', 'HKD', '0.04', '0.015', '0.027', '1,312,500']
    # li = ['85987', 'CCB', 'B2304-R', 'CNY', 'N/A', '-', '-', '-']
    
    code = int(li_1[0])

    nos  = ' '.join(li_1[1:-5])                   # nos = NAME OF STOCK
    cur  = li_1[-5]

    cpc  = check(li_1[-4])                   # cpc = CUR PRV.CLO. 
    cls = check(li_2[0])                # cls = CLOSING

    ask  = check(li_1[-3])  
    bid = check(li_2[-3])

    high = check(li_1[-2]) 
    low = check(li_2[-2])

    st = check_lsit_last_item(li_1[-1])     # st = SHARES TRADED
    tnvr = check_lsit_last_item(li_2[-1])       # tnvr = TURNOVER


    

    return code, nos, cur, cpc, cls, ask, bid, high, low, st, tnvr


In [219]:
# total item in quotations_lists
total_item = len(quotations_lists)

final_list = []
i = 0
while (i< total_item):

    item_1= quotations_lists[i]
    list_1 = splited_list(item_1)

    if '*' in list_1:
        list_1.remove('*')
        
    if 'SUSPENDED' in list_1:
        dic = {}
        code, nos, cur = suspended(list_1)
        dic['CODE'] = code
        dic['NAME OF STOCK'] = nos
        dic['CUR'] = cur

        final_list.append(dic)
        i+=1

    if 'HALTED' in list_1:
        dic = {}
        code, nos, cur = suspended(list_1)
        dic['CODE'] = code
        dic['NAME OF STOCK'] = nos
        dic['CUR'] = cur

        final_list.append(dic)
        i+=1


    if 'SUSPENDED' not in list_1:
        item_2 = quotations_lists[i+1]
        list_2 = splited_list(item_2)
        
        dic = {}
        
        code, nos, cur, cpc, cls, ask, bid, high, low, st, tnvr = not_suspended(list_1, list_2)

        dic['CODE'] = code
        dic['NAME OF STOCK'] = nos
        dic['CUR'] = cur
        dic['CUR PRV.CLO.'] = cpc
        dic['CLOSING'] = cls
        dic['ASK'] = ask
        dic['BID'] = bid
        dic['HIGH'] = high
        dic['LOW'] = low
        dic['SHARES TRADED'] = st
        dic['TURNOVER'] = tnvr

        final_list.append(dic)
        i +=2

In [220]:
final_list[-1]

{'CODE': 87001,
 'NAME OF STOCK': 'HUI XIAN REIT',
 'CUR': 'CNY',
 'CUR PRV.CLO.': 1.02,
 'CLOSING': 1.02,
 'ASK': 1.02,
 'BID': 1.01,
 'HIGH': 1.02,
 'LOW': 1.01,
 'SHARES TRADED': 1366091,
 'TURNOVER': 1381649}

In [221]:
len(final_list)

16888

In [222]:
# Creating Data Frame using pandas
df = pd.DataFrame(final_list)

In [223]:
df

,CODE,NAME OF STOCK,CUR,CUR PRV.CLO.,CLOSING,ASK,BID,HIGH,LOW,SHARES TRADED,TURNOVER
0,1,CKH HOLDINGS,HKD,54.85,54.35,54.35,54.3,55.4,54.25,5234462,286201169
1,2,CLP HOLDINGS,HKD,76.95,76.5,76.55,76.5,77.3,76.4,3374157,258528175
2,3,HK & CHINA GAS,HKD,8.74,8.52,8.53,8.52,8.76,8.51,30744940,264009852
3,4,WHARF HOLDINGS,HKD,25.15,25.25,25.25,25.2,25.4,24.95,683860,17249090
4,5,HSBC HOLDINGS,HKD,51.75,51.25,51.3,51.25,52.1,51.0,20744261,1069075134
...,...,...,...,...,...,...,...,...,...,...,...
16883,86657,PRC B2207-R,CNY,N/A,N/A,-,-,-,-,-,-
16884,86658,PRC B2507-R,CNY,N/A,N/A,-,-,-,-,-,-
16885,86659,PRC B2309-R,CNY,N/A,N/A,-,-,-,-,-,-
16886,86660,PRC B2609-R,CNY,N/A,N/A,-,-,-,-,-,-


In [224]:
# exporting data frame to CSV file
df.to_csv('all_data.csv',index=False)